In [1]:
pip install -U scikit-image

     |████████████████████████████████| 13.5 MB 97 kB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
!pip install skan

     |████████████████████████████████| 1.5 MB 5.8 MB/s 
     |████████████████████████████████| 51 kB 44 kB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Necessary Packages

In [5]:
import cv2
import os
import skan
import skimage.io
import glob
import numpy as np
import pandas as pd
from pathlib import Path
from skimage import measure 
from skimage import morphology
import matplotlib.pyplot as plt
from skimage.measure import label
from numpy.core.numeric import False_
from skimage.morphology import skeletonize
from pandas._libs.lib import fast_unique_multiple_list_gen
from skimage.morphology import (erosion, dilation, closing, opening,
                                area_closing, area_opening)

# Automatic Thresholding Algorithm

In [7]:
#used to segment RGB images (pred image)
def get_segment_pred(filename):

      # read the original image
      img = cv2.imread(filename)

      #extracting red channel
      img = img[:,:,2] 

      #retain only stem filename (number of img)
      pred_name = Path(filename).stem

      # median blur before thresholding
      blurred_img = skimage.filters.median(img)

      # perform automatic thresholding to produce a binary image
      t = skimage.filters.threshold_otsu(blurred_img) ##for Li's Method replace line w/: t = skimage.filters.threshold_li(blurred_img)
      binary_mask = blurred_img > t

      #creating border mask 
      border_mask = np.ones(shape=binary_mask.shape[0:2], dtype="bool")

      rr, cc = skimage.draw.rectangle(start=(50,185), end=(1870, 1652))
      border_mask[rr, cc] = False

      binary_mask[border_mask] = 0

      binary_mask = dilation(binary_mask)

      ##Finding largest connected component 

      #image labeling 
      label_img = measure.label(binary_mask, connectivity = img.ndim)

      largestCC = label_img == np.argmax(np.bincount(label_img.flat, weights=binary_mask.flat))
      
      pred_final_img = largestCC.astype(np.uint8)


      return pred_final_img, pred_name

In [6]:
#used for ground truth (simple reading of binary images)
def get_ground_truth(filename):

      # read the original image
      img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

      true_name = Path(filename).stem

      true_final_img = img.astype(np.uint8)

      return true_final_img, true_name

In [8]:
# retrives measurements of predicted and GT images

def get_measurements(pred_final_img, true_final_img, pred_name, true_name):

      scale = 0.14

      #skeletonize pred_final_img
      skeleton_p = skeletonize(pred_final_img>0)
      skeleton_csr_p = skan.csr.Skeleton(skeleton_p)

      df_p = skan.summarize(skeleton_csr_p)

      pred_branch = df_p[df_p['branch-type'] == 1] 
      pred_branch_type_num = len(pred_branch['branch-type'])
      pred_branch_distance = pred_branch['branch-distance']
      pred_branch_mean = pred_branch_distance.mean() 
      pred_branch_max = pred_branch_distance.max() 
      pred_branch_std = pred_branch_distance.std() 

      #skeletonize true_final_img
      skeleton_t = skeletonize(true_final_img>0)
      skeleton_csr_t = skan.csr.Skeleton(skeleton_t)

      df_t = skan.summarize(skeleton_csr_t)

      true_branch = df_t[df_t['branch-type'] == 1] 
      true_branch_type_num = len(true_branch['branch-type'])
      true_branch_distance = true_branch['branch-distance']
      true_branch_mean = true_branch_distance.mean() 
      true_branch_max = true_branch_distance.max() 
      true_branch_std = true_branch_distance.std() 


      #calculating measurements via region props
      pred_props = measure.regionprops_table(pred_final_img, properties = ['area', 'perimeter', 'feret_diameter_max', 'solidity', 'convex_area', 'major_axis_length'])

      pred_props["Dice"] = dice_coef(pred_final_img, true_final_img)
      pred_df = pd.DataFrame(pred_props)  
      
      #calculating measurements via region props
      true_props = measure.regionprops_table(true_final_img, properties = ['area', 'perimeter', 'feret_diameter_max', 'solidity', 'convex_area', 'major_axis_length'])
      true_df = pd.DataFrame(true_props)  

      #converting to micron scale
      pred_df['area'] = pred_df['area'] * (scale**2) #(scale_x * scale_y) 
      pred_df['perimeter'] = pred_df['perimeter'] * (scale)
      pred_df['feret_diameter_max'] = pred_df['feret_diameter_max'] * (scale)
      pred_df['convex_area'] = pred_df['convex_area'] * (scale**2)
      pred_df['major_axis_length'] = pred_df['major_axis_length'] * (scale)

      pred_df["branch_number"] = pred_branch_type_num
      pred_df["branch_mean"] = pred_branch_mean * scale
      pred_df["branch_max"] = pred_branch_max * scale
      pred_df["branch_std"] = pred_branch_std * scale

      pred_df.insert(0, 'Name', pred_name)


      true_df['area'] = true_df['area'] * (scale**2) #(scale_x * scale_y) 
      true_df['perimeter'] = true_df['perimeter'] * (scale)
      true_df['feret_diameter_max'] = true_df['feret_diameter_max'] * (scale)
      true_df['convex_area'] = true_df['convex_area'] * (scale**2)
      true_df['major_axis_length'] = true_df['major_axis_length'] * (scale)

      true_df["branch_number"] = true_branch_type_num
      true_df["branch_mean"] = true_branch_mean * scale
      true_df["branch_max"] = true_branch_max * scale
      true_df["branch_std"] = true_branch_std * scale

      true_df.insert(0, 'Name', true_name)
      
      return pred_df, true_df

# Dice Coefficient (Evaluation Metric)

In [9]:
import scipy.sparse
from scipy.spatial import distance

# computes similarity values between prediction and GT

def dice_coef(pred_final_image, true_final_image, smooth=1):
  true_f = true_final_image.flatten().astype(dtype=bool)

  pred_f = pred_final_image.flatten().astype(dtype=bool)

  dice = distance.dice(pred_f, true_f)
  dice = 1 - dice
  return dice

# Run on Full Dataset + Saving Segmentations

In [10]:
from pandas._libs.tslibs.period import get_period_field_arr

#run algorithm on all images
pred_dfs =[]
true_dfs =[]

files = sorted(glob.glob("/content/drive/MyDrive/Colab_Notebooks/SULI2022/Root_photography/Root_Image_Dataset_1/*.*"))

#files.sort()
for filename in files:
  try:
    filename_mask = "/content/drive/MyDrive/Colab_Notebooks/SULI2022/Root_photography/Root_Image_Dataset_1_GT/" + Path(filename).stem + ".png" #os.path.basename(filename) 

    pred_img, pred_name = get_segment_pred(filename) #==> returns a pred image np array and shortened name

    true_img, true_name = get_ground_truth(filename_mask) #==> returns a true image array

    pred_df, true_df = get_measurements(pred_img, true_img, pred_name, true_name)

    pred_dfs.append(pred_df)
    true_dfs.append(true_df)

    ##1 - distance dice = similarity 
    print(pred_name, true_name, dice_coef(pred_img,true_img))

    ## saving predicted binary segmenation
    path = '/content/drive/MyDrive/Pred_Segmentations'
    cv2.imwrite(os.path.join(path, pred_name + '.jpg'), pred_img*255)
    #print('image is saved Successfully')
    cv2.waitKey(0)

  except Exception:
    print('error:', pred_name, true_name)
       
pred_dfs = pd.concat(pred_dfs)
true_dfs = pd.concat(true_dfs)

pred_dfs.to_csv (r'/content/drive/MyDrive/PRED_Segmentation_Results.csv', index = False, header=True)
true_dfs.to_csv (r'/content/drive/MyDrive/GT_Segmentation_Results.csv', index = False, header=True)

100_104 100_104 0.7716462874357611
100_120 100_120 0.7156120638675341
100_130 100_130 0.8054751250329034
100_193 100_193 0.7390012274694748
100_20 100_20 0.8246321411740064
100_210 100_210 0.8051287032540068
100_219 100_219 0.8184126098479565
100_226 100_226 0.7546763558067555
100_244 100_244 0.8019901449552695
100_254 100_254 0.8273407651645994
100_27 100_27 0.7753327465258328
100_304 100_304 0.8332762572113485
100_315 100_315 0.8001239267062052
100_328 100_328 0.8195496180681856
100_336 100_336 0.7890906343650055
100_35 100_35 0.7219512195121951
100_355 100_355 0.8017861391212049
100_468 100_468 0.7966008166868999
100_660 100_660 0.8536407494670706
10_134 10_134 0.7885471253892307
10_138 10_138 0.8008515313830518
10_142 10_142 0.7107618343195266
10_143 10_143 0.7789555675968572
10_145 10_145 0.769109732329644
10_149 10_149 0.8370402053036784
10_154 10_154 0.7917598640064418
10_215 10_215 0.7657883730656629
10_219 10_219 0.8398698557861414
10_25 10_25 0.8088321500683906
10_265 10_265 

In [11]:
#w/ dilation + median
data = pd.read_csv('/content/drive/MyDrive/PRED_Segmentation_Results.csv')
data['Dice'].mean()

0.7625100663749323